In [ ]:
# Predict on an image
sample_image_path = "D:/Testing (Triya ma'am)/Hematoxylin eosin staining for scoring at 20X and 40x/NASH_40X/4.jpeg"
img = tf.keras.preprocessing.image.load_img(sample_image_path, target_size=(img_width, img_height))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match batch size
img_array /= 255.  # Normalize image

prediction = model.predict(img_array)

In [ ]:
# Get the index of the class with maximum probability
max_prob_index = np.argmax(prediction)
    
# Define the NASH score mapping
nash_scores = {
        0: "0: No foci per 200× field",
        1: "1: <2 foci or 2-4 foci per 200×field",
        2: "2: >4 foci per 200×field"
    }
    
# Get the NASH score corresponding to the maximum probability
nash_score = nash_scores[max_prob_index]

In [ ]:
# Load the sample image
sample_img = cv2.imread(sample_image_path)
sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)

# Resize the image to match the model's input size
resized_sample_img = cv2.resize(sample_img, (img_width, img_height))

# Convert the image to grayscale
gray_img = cv2.cvtColor(resized_sample_img, cv2.COLOR_RGB2GRAY)

In [ ]:
# Thresholding
_, thresh_img = cv2.threshold(gray_img, 100, 255, cv2.THRESH_BINARY)

plt.imshow(thresh_img)

# Find contours
contours, _ = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Get the contour with the maximum area
max_contour = max(contours, key=cv2.contourArea)

# Get the bounding box of the maximum contour
x, y, w, h = cv2.boundingRect(max_contour)

# Define the size of the 200x area
area_size = 200  # Assuming the area is square

# Calculate the coordinates for the 200x area
x_center, y_center = x + w // 2, y + h // 2
x_200x = max(0, x_center - area_size // 2)
y_200x = max(0, y_center - area_size // 2)
w_200x = min(area_size, resized_sample_img.shape[1] - x_200x)
h_200x = min(area_size, resized_sample_img.shape[0] - y_200x)

# Overlay predicted NASH score on the image
annotated_image = resized_sample_img.copy()
cv2.putText(annotated_image, f"Predicted NASH Score: {nash_score}", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

# Draw a rectangle around the 200x area
cv2.rectangle(annotated_image, (x_200x, y_200x), (x_200x + w_200x, y_200x + h_200x), (0, 255, 0), 2)

# Plot the annotated image
plt.figure(figsize=(8, 6))
plt.imshow(annotated_image)
plt.title("Image with Annotations")
plt.axis('off')
plt.show()

# Print the NASH score
print("Prediction:", prediction)
print('Predicted NASH score:', nash_score)